# <font color=blue>Итераторы и генераторы</font>

## <font color=green>Итераторы</font>

Итак, чем же является итератор? Итератор — это вспомогательный объект, возвращающий следующий элемент всякий раз, когда выполняется вызов функции `next()` с передачей этого объекта в качестве аргумента. Таким образом, любой объект, предоставляющий метод `__next__()`, является итератором, возвращающим следующий элемент при вызове этого метода, при этом совершенно неважно как именно это происходит.

Итак, итератор — это некая фабрика по производству значений. Всякий раз, когда вы обращаетесь к ней с просьбой "давай следующее значение", она знает как сделать это, поскольку сохраняет своё внутреннее состояние между обращениями к ней.

Существует бесчисленное множество примеров использования итераторов. Например, все функции пакета itertools возвращают итераторы. 

Некоторые из которых генерируют бесконечные последовательности:

In [1]:
from itertools import count

counter = count(start=13)
print(next(counter))
print(next(counter))

13
14


В следующем примере генерируются все возможные перестановки элементов из `L`.

In [3]:
from itertools import permutations

L = list(range(4))

perms = permutations(L)

for _ in range(10):
    print(next(perms))

(0, 1, 2, 3)
(0, 1, 3, 2)
(0, 2, 1, 3)
(0, 2, 3, 1)
(0, 3, 1, 2)
(0, 3, 2, 1)
(1, 0, 2, 3)
(1, 0, 3, 2)
(1, 2, 0, 3)
(1, 2, 3, 0)


Итератор останавливается, когда выбрасывается исключение `StopIteration`

In [7]:
it = iter([1, 2])

print(next(it))
print(next(it))
print(next(it))

1
2


StopIteration: 

Итераторы в Python применяются повсеместно:

1. При использовании списка `for` объект, по которому выполняется итерация преобразуется в итератор, то есть код

```python
for a in obj:
    do_smth()
```

реализуется приблизительно так

```python
it = iter(obj)
while True:
    try:
        a = next(it)
    except StopIteration:
        break
    do_smth(a)
```

2. `map`-, `zip`-, `enumerate`-, `filter`-, `reversed`- объекты являются итераторами.

3. Файловые объекты являются итераторами.

### <font color=red>Важно!</font>
По итератору можно выполнить тольк один проход. Для повторной итерации нужно создавать новый итератор.

In [10]:
e = enumerate([1, 2])
print(next(e))
print(next(e))
next(e)

(0, 1)
(1, 2)


StopIteration: 

In [12]:
for i in e:
    print(i)

In [13]:
e = enumerate([1, 2])
print(next(e))
print('loop')
for i in e:
    print(i)

(0, 1)
loop
(1, 2)


## <font color=green>Создание своих итераторов</font>

1. В первую очередь у объекта-итератора должен быть метод `__next__()`, возвращающий очередной элемент.

2. Если элементы закончились, объект должен бросать исключение `StopIteration`.

3. У итератора должен быть метод `__iter__()`. Метод `__iter__()` при примении к объекту встроенной функции `iter()` и возвращает сам итератор. Система `iter()` - `__iter__()` итераторов из контейнеров (списков, словарей, кортежей, множеств) и строк и в итераторах нужна для совместимости интерфейса.

### Пример 1. Единицы

Итератор, создающий заданное количество единиц.

In [15]:
class Ones:
    def __init__(self, n):
        self._n = n
        self._counter = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self._counter < self._n:
            self._counter += 1
            return 1
        else: 
            raise StopIteration()
        
        
ones = Ones(3)
for k in ones:
    print(k)

1
1
1


### Упражнение 1. Числа Фибоначчи

Напишите итератор, для первых `n` чисел Фибоначчи.

In [20]:
class FibbIter:
    def __init__(self, n):
        self.n = n
        self.counter = 0
        self.fibb = (1, 1)
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.counter >= self.n:
            raise StopIteration()
            
        if self.counter == 0 or self.counter == 1:
            self.counter += 1
            return 1
        else:
            self.fibb = self.fibb[1], self.fibb[0] + self.fibb[1]
            self.counter += 1
            return self.fibb[1]

In [22]:
fibb = FibbIter(10)

for i in fibb:
    print(i)

1
1
2
3
5
8
13
21
34
55


### Упражнение 2. Чтение из файла

Напишите итератор считывающий из файла по 10 символов. При последнем считывании, если невозможно вернуть 10 символов, итератор возращает столько, сколько есть.

In [42]:
class ReadIter:
    def __init__(self, path):
        self.file = open(path, 'r')
        
    def __iter__(self):
        return self
    
    def __next__(self):
        next_read = self.file.read(10)
        if next_read == '':
            self.file.close()
            raise StopIteration()
        return next_read

In [44]:
reader = ReadIter('text.txt')
for i in reader:
    print(repr(i))

'hello,text'
',more text'
', one more'
' string, 1'
'234567890'


## <font color=green>Генераторы</font>

Генераторы - это разновидность итераторов. Их отличает краткость и удобство реализации, благодаря чему они создаются разработчиками гораздо чаще. 

Генератор – это функция, которая будучи вызванной в функции `next()` возвращает следующий объект согласно алгоритму ее работы. Вместо ключевого слова `return` в генераторе используется `yield`. Проще всего работу генератор посмотреть на примере. Напишем функцию, которая генерирует необходимое нам количество единиц.

In [45]:
def ones(n):
    for _ in range(n):
        yield 1
        
        
my_ones = ones(2)
print(next(my_ones))
print(next(my_ones))
print(next(my_ones))

1
1


StopIteration: 

In [46]:
my_ones = ones(5)

for a in my_ones:
    print(a)

1
1
1
1
1


In [47]:
my_ones = iter(ones(2))
print(dir(my_ones))

['__class__', '__del__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__name__', '__ne__', '__new__', '__next__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'close', 'gi_code', 'gi_frame', 'gi_running', 'gi_yieldfrom', 'send', 'throw']


При выполнении инструкции `yield` весь контекст функции (локальные переменные) сохраняются. При следующем примении `next()` к генератору функция продолжает, как ни в чем не бывало, работать с инструкции, следующей за `yield`.

Важно отметить, что тело генератора в первый раз выполняется при примении к нему `next()`, а не при его создании.

In [48]:
def gen():
    print("I am generator")
    yield None
    
g = gen()
print("And now next() is called")
next(g)

And now next() is called
I am generator


###  Упражнение 3.

Реализуйте итераторы из упражнений 1 и 2 в виде генераторов.

In [55]:
def fibb_gen(n):
    fibb = 1, 1
    
    for i in range(n):
        if i < 2:
            yield 1
        else:
            fibb = fibb[1], fibb[0] + fibb[1]
            yield fibb[1]

In [56]:
fib = fibb_gen(10)
for i in fib:
    print(i)

1
1
2
3
5
8
13
21
34
55


In [ ]:
def read_gen()

In [ ]:
class ReadIter:
    def __init__(self, path):
        self.file = open(path, 'r')
        
    def __iter__(self):
        return self
    
    def __next__(self):
        next_read = self.file.read(10)
        if next_read == '':
            self.file.close()
            raise StopIteration()
        return next_read